In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util

import getpass
import logging

# Read artists

In [64]:
file = open("../data/artists.txt","r") 

In [65]:
u_artists = [x.replace("\n", "") for x in file.readlines()]

In [66]:
file.close() 

In [67]:
len(u_artists)

49

# Get Spotify Access

In [5]:
"""create a log for errors"""
logger = logging.getLogger('ezooplaylist')
hdlr = logging.FileHandler('../out/playlists.log')
formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
hdlr.setFormatter(formatter)
logger.addHandler(hdlr)
logger.setLevel(logging.INFO)

In [6]:
"""get access from spotify"""
sp = spotipy.Spotify()
#get access info
cid = getpass.getpass()
secret = getpass.getpass()
logger.info("got spotify access info")
#establish access
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
logger.info("got spotify access")

········
········


In [7]:
"""find an aritist's top spotify songs uri(s) based on the artist's spotify id"""
def get_top_songs_for_artist(artist, song_count=5):
    """find an aritist's top spotify songs uri(s) based on the artist's spotify id.
    Args:
        artist (str): name of an artist
        song_count (int): number of top songs you want to find
    Returns:
        song_ids (list): top n song uris
    """
    song_ids = []
    try:
        t = sp.search("artist:{}".format(artist), limit = 1)
        artist_id = t['tracks']['items'][0]['artists'][0]['id']
        artist_top_tracks = sp.artist_top_tracks(artist_id)
        artist_top_tracks_length = len(artist_top_tracks['tracks'])
        for i in range(0, artist_top_tracks_length if song_count > artist_top_tracks_length else song_count ):
            song_ids.append(artist_top_tracks['tracks'][i]['uri'])
            logger.info(str(len(song_ids)) + ' songs found - ' + artist)
    except:
        logger.info('Artist not found - ' + artist)
    return song_ids

In [8]:
"""input user name"""
username = getpass.getpass()

········


In [18]:
"""get token for user authentication"""
token = util.prompt_for_user_token(username, 
                                   scope='playlist-modify-private,playlist-modify-public', 
                                   client_id = cid, 
                                   client_secret=secret,
                                   redirect_uri='http://localhost:8888/callback')

In [19]:
"""establish connection"""
sp = spotipy.Spotify(auth=token)

In [11]:
playlists = sp.user_playlists(username)

In [12]:
for p in playlists['items']:
    if p['name'] == 'Electric Zoo 2019':
        p_id = p['id']

In [13]:
if not 'p_id' in globals():
    sp.user_playlist_create(username, "Electric Zoo 2019")
    for p in playlists['items']:
        if p['name'] == 'Electric Zoo 2019':
            p_id = p['id']

In [20]:
e_lst = sp.user_playlist(username, p_id)

In [38]:
if len(e_lst['tracks']['items']) > 0:
    exist_song_ids = []
    for item in e_lst['tracks']['items']:
        exist_song_ids.append(item['track']['uri'])

In [44]:
api_track_add_limit = 100

In [45]:
all_track_ids = []
for current_artist in u_artists:
    top_song_limit_per_artist = 4
    top_artist_songs = get_top_songs_for_artist(current_artist, top_song_limit_per_artist)
    if 'exist_song_ids' in globals():
        all_track_ids.extend([x for x in top_artist_songs if x not in exist_song_ids])
    else:
        all_track_ids.extend(top_artist_songs)

In [46]:
all_songs_len = len(all_track_ids)

In [48]:
sp.user_playlist_add_tracks(user=username, 
                            playlist_id=p_id, 
                            tracks=all_track_ids[:min(api_track_add_limit, all_songs_len)])

{'snapshot_id': 'Myw0ZTRiYzFlZTI4YTM3M2Y5Yjc3NTgzMzJiNzY0M2E3ZjQ2MzhhM2Rm'}